# Data challenge 

## Auteurs : Sarah BAKHOUCHE, Razan ALTUJJAR & Wilfrid NKODO

In [5]:
%matplotlib inline
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix
#--------------------------


from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from time import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta

In [7]:
X_train=pd.read_csv("X_train.csv")
Y_train=pd.read_csv("y_train.csv")
X_test=pd.read_csv("X_test.csv")
Y=pd.read_csv("y.csv")

print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y))

(37368, 27)
(37368, 2)
(9339, 27)
(9339, 2)


In [8]:
#métric pour le calcul du score 
from sklearn.metrics import mean_absolute_percentage_error

def custom_metric_function(y_true, y_pred):
    return mean_absolute_percentage_error(y_true.price, y_pred.price)*100

# Traitement des variables

## Traitement du jeu de données d'apprentissage X_train

Dans un premier temps , on obeserve les types de variables contenues dans notre jeu de données. Nous traiterons ensuite de prime abord les variables de type objet les variables de type numérique

In [9]:
print(X_train.dtypes) #observation du type des variables 
X_train

id_annonce                       int64
property_type                   object
approximate_latitude           float64
approximate_longitude          float64
city                            object
postal_code                      int64
size                           float64
floor                          float64
land_size                      float64
energy_performance_value       float64
energy_performance_category     object
ghg_value                      float64
ghg_category                    object
exposition                      object
nb_rooms                       float64
nb_bedrooms                    float64
nb_bathrooms                   float64
nb_parking_places              float64
nb_boxes                       float64
nb_photos                      float64
has_a_balcony                  float64
nb_terraces                    float64
has_a_cellar                   float64
has_a_garage                   float64
has_air_conditioning           float64
last_floor               

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors
0,35996577,appartement,43.643880,7.117183,villeneuve-loubet,6270,63.0,NaN,NaN,NaN,...,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,35811033,appartement,45.695757,4.895610,venissieux,69200,90.0,3.0,NaN,223.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,35731841,maison,47.966791,-1.220451,moutiers,35130,61.0,NaN,370.0,NaN,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35886765,maison,47.289292,-1.878805,cordemais,44360,142.0,NaN,764.0,217.0,...,0.0,1.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,35781137,appartement,45.718992,4.844234,lyon-7eme,69007,88.0,3.0,NaN,NaN,...,0.0,1.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,35881571,appartement,43.480292,-1.484324,bayonne,64100,2995.0,NaN,NaN,255.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37364,35890109,appartement,48.861391,2.401158,paris-20eme,75020,36.0,4.0,NaN,229.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37365,36010245,appartement,44.397837,-1.164312,biscarrosse,40600,99.0,2.0,NaN,NaN,...,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
37366,35864579,duplex,48.864204,2.808693,magny-le-hongre,77700,9281.0,NaN,NaN,NaN,...,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Nous remarquons que les variables présentent différents types : numeriques et objet

In [10]:
# traitement des NAN des variables type objet

X_train[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]] = X_train[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]].replace(np.nan,"unknown")
from sklearn import preprocessing 
label = preprocessing.LabelEncoder() 
X_train['property_type' ]=label.fit_transform(X_train['property_type'])
X_train['city' ]=label.fit_transform(X_train['city'])
X_train['energy_performance_category' ]=label.fit_transform(X_train['energy_performance_category'])
X_train['ghg_category']=label.fit_transform(X_train['ghg_category'])
X_train['exposition']=label.fit_transform(X_train['exposition'])


Parmi les variables de type numérique, on remarque des valeurs manquantes. Au lieu de les supprimer et par conséquent perdre des informations qui peuvent s'avérer importantes, on les remplace par les médiane de la variable numérique correspondante pour la poursuite de notre étude. 

In [12]:
#traitement des variables type numérique 
import statistics

X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']] = X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].replace(np.nan,X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].median())
X_train


,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors
0,35996577,0,43.643880,7.117183,8452,6270,63.0,2.0,795.0,180.0,...,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,35811033,0,45.695757,4.895610,8234,69200,90.0,3.0,795.0,223.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,35731841,12,47.966791,-1.220451,5235,35130,61.0,2.0,370.0,180.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35886765,12,47.289292,-1.878805,1929,44360,142.0,2.0,764.0,217.0,...,0.0,1.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,35781137,0,45.718992,4.844234,4476,69007,88.0,3.0,795.0,180.0,...,0.0,1.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,35881571,0,43.480292,-1.484324,592,64100,2995.0,2.0,795.0,255.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37364,35890109,0,48.861391,2.401158,5617,75020,36.0,4.0,795.0,229.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37365,36010245,0,44.397837,-1.164312,827,40600,99.0,2.0,795.0,180.0,...,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
37366,35864579,6,48.864204,2.808693,4502,77700,9281.0,2.0,795.0,180.0,...,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Traitement du jeu de données test X_test

On va traiter les variables de type objet et les variables de type numérique de la même facon

In [13]:
import statistics
from sklearn import preprocessing 
X_test[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]] = X_test[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]].replace(np.nan,"unknown")

label = preprocessing.LabelEncoder() 
X_test['property_type' ]=label.fit_transform(X_test['property_type'])
X_test['city' ]=label.fit_transform(X_test['city'])
X_test['energy_performance_category' ]=label.fit_transform(X_test['energy_performance_category'])
X_test['ghg_category']=label.fit_transform(X_test['ghg_category'])
X_test['exposition']=label.fit_transform(X_test['exposition'])

X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']] = X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].replace(np.nan,X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].median())
X_test

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors
0,35160615,10,49.371670,3.533991,470,2220,143.0,2.0,800.0,299.0,...,0.0,1.0,11.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,35830639,14,49.037190,-0.982385,3525,50160,470.0,2.0,28559.0,180.0,...,0.0,1.0,20.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,36016657,0,47.097180,6.149253,2436,25290,135.0,2.0,780.5,180.0,...,0.0,0.0,6.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,35759225,0,43.699670,7.245556,2358,6000,3637.0,3.0,780.5,210.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35252229,4,43.849435,6.215461,2296,4360,600.0,2.0,780.5,180.0,...,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9334,36052217,19,43.117121,5.790808,3307,83110,96.0,2.0,780.5,180.0,...,1.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9335,35823719,0,43.099598,5.874048,1574,83500,71.0,3.0,780.5,51.0,...,0.0,1.0,8.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
9336,35793053,0,45.977895,-1.096506,1195,17450,70.0,2.0,780.5,180.0,...,1.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9337,36049283,10,47.284067,-2.197979,3144,44600,120.0,2.0,362.0,65.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Méthodes d'apprentissage sur les données tabulaires


In [14]:
#découpage en echantillon train et échantillon test 
from sklearn.model_selection import train_test_split



X_train,X_test = train_test_split(X_train, test_size = 0.40)  
Y_train,Y_test = train_test_split(Y_train, test_size = 0.40)  

Y_train=Y_train['price']
Y_test=Y_test['price']
Y_train=pd.DataFrame(Y_train)
Y_test=pd.DataFrame(Y_test)

## Regression linéaire

In [15]:
reg=LinearRegression().fit(X_train,Y_train)
pred=reg.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']

#score
custom_metric_function(Y_test, pred) # 117.49632146661118

118.96903590076941

## Kneighbors


In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)
pred=knn.predict(X_test)

pred=pd.DataFrame(pred)
pred.columns=['price']

#score
custom_metric_function(Y_test, pred) # 69.0995077735031

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


69.0995077735031

## Random Forest

paramètres : 
- criterion : l'erreur de notre modèle sera calcule grâce à l'indice de Gini, criterion ='Gini'
- n_estimators : nombre d'arbres dans notre foret, plus il est grand plus le modele est précis mais plus le modèle est long. (+++)
- max_depth : important. détermine la longueur maximum de l'arbre. généralement on prend des arbres peu profonds. (+++)
- max_leaf_nodes : le nombre de feuiles maximal de l'arbre. Il peut être intéressant de fixer le nombre de feuilles, il garde bien sur les feuilles maximisant les performances de l'arbre. olus il est grand et plus le modèle tend vers le surapprentissage. (+++)
- bootstrap : toujours true.  
- oobs : estime la généralisation du modèle en testant sur les parties du jeu de données qui n'a pas été sélectionné pour sa construction. utile pour vérifier la généralisation du modèle 
- oobs_score : mettre true pour avoir le score oob
- n_jobs : nombre de processuers alloué au modèle. plus il est grand, plus l'entrainement est rapide.
- random state : important en phase de test car permet de fixer l'aléatoire au préalable. utilse pour comparer les performances qui diffèrent par un changment de paramètre et non par un alétoire de départ différent. 
- verbose : nombre d'infos affichées. on garde verbose = 0

+++ paramètre à optimiser 


atributs (importants): 
- .feature_importance_ : retourne l'importance des features du jeu de données dans la construction de la forêt. 
- .n_classe : nombre de classes 

In [ ]:
#utiliser grid_search pour chaque paramètre il test différents valeurs et ressort le modèle le plus performants pour chacun des paramètre 

param_grid = {'n_estimators':[5,10,20,30,40,50,100], 
'max_features':range(2,27),
'max_leaf_nodes' : [2,4,6,8,10,15,20,30,50,100,200,300,400,500,800]}


rf = RandomForestRegressor(random_state=123, verbose=0)

# grid search using cv
rf_model = GridSearchCV(rf, param_grid)
rf_model.fit(X_train, Y_train)

In [ ]:
#on veut voir les meilleurs estimateurs 

best_random = rf_model.best_estimator_
best_random

RandomForestRegressor(max_features=14, max_leaf_nodes=20, n_estimators=10,
                      random_state=123)

In [ ]:
pred=rf_model.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']
custom_metric_function(Y_test, pred)

291.1809413609381

In [15]:
from sklearn.ensemble import RandomForestRegressor
maxnodes = [2,4,6,8,10,15,20,30,50,100,200,300,400,500,800]
maxfeatures=range(2,10)

oob_error = []

for maxfe in maxfeatures:
    RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)


for maxno in maxnodes:
    RF = RandomForestRegressor(n_estimators=50,max_leaf_nodes=maxno,oob_score = True).fit(X_train, Y_train)

/tmp/ipykernel_94/2771373303.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_94/2771373303.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_94/2771373303.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_94/2771373303.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

In [19]:
pred=RF.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']

custom_metric_function(Y_test, pred) #120.96478424598507

120.96478424598507

## XGboost

On va tester une méthode XGbBoost

In [20]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import xgboost as xgb

xgb=XGBRegressor(objective ='reg:linear',verbosity=0,base_score=1,n_estimators=1000,booster='gblinear',max_depth=5,learning_rate=0.1,loss="mean_absolute_error")
xgb.fit(X_train, Y_train)
xgb_pred=xgb.predict(X_test)


In [21]:
xgb_pred = pd.DataFrame(xgb_pred)
xgb_pred.columns=['price']

#score
custom_metric_function(Y_test, xgb_pred)  # 118.88702558392283

118.88702558392283

## Réseau de Neurones

In [22]:
modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)

training = modele.fit (X_train, Y_train, epochs = 70, validation_split = 0.1)

#historique = pd.DataFrame(training.history)
#historique['epoque'] = training.epoch
#figure, axe = plt.subplots(figsize = (14,8))
#num_epoque = historique.shape[0]
#axe.plot(np.arange(0, num_epoque), historique["mae"],
#label = "Training MAE", lw = 3, color = 'red')
#axe.plot(np.arange(0, num_epoque), historique["val_mae"],
#label = "Validation MAE", lw = 3, color = 'blue')
#axe.legend ()
#plt.tight_layout()
#plt.show()

#test1 = test.iloc[[10]]
test_prediction = modele.predict(X_test).squeeze()
#test_label = test_cible.iloc[10]
#print("Prédiction du modèle = {:.2f}".format(test_prediction))
#print("Valeur actuelle = {:.2f}".format(test_label))

#score = modele.evaluate( X_test, Y_test, verbose = 0)
#print(score)

pred=pd.DataFrame(test_prediction)

pred=pd.DataFrame(pred)
pred.columns=['price']
pred

custom_metric_function(Y_test, pred) #61.78658970032733

2022-12-13 13:57:56.012004: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-13 13:57:56.012045: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-13 13:57:56.012067: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-3c590c73-ebe5-4e60-8b74-f7ba67114dc0): /proc/driver/nvidia/version does not exist
2022-12-13 13:57:56.012402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Epoch 1/70
631/631 [==============================] - 3s 4ms/step - loss: 68

61.78658970032733

# Traitement des images 

On va importer les images disponibles dans le dossier. 

In [ ]:
import cv2
import numpy as np
from skimage.filters import sobel

img_test = []
les_index_test = []

for dossier in data_test_list:
    les_images = os.listdir(data_path_test+'/'+dossier)
    les_values = [] # Liste pour ranger les listes de valeurs d'intérêt
    for img in les_images:
        values = [] # Liste des valeurs d'intérêt pour cette image
        # Image en couleur
        a_raw = cv2.imread(data_path_test+'/'+dossier+'/'+img)
        values.append(a_raw[:,:,0].mean()) # Valeur moyenne de bleu sur l'image
        values.append(a_raw[:,:,1].mean()) # Valeur moyenne de vert sur l'image
        values.append(a_raw[:,:,2].mean()) # Valeur moyenne de rouge sur l'image
        # Image en nuances de gris
        a_grey = cv2.cvtColor(a_raw, cv2.COLOR_BGR2GRAY)
        values.append(a_grey.sum()) # Somme des valeurs de gris des pixels de l'image
        values.append(np.percentile(a_grey, 25)) # Premier quartile des valeurs de gris des pixels de l'image
        values.append(np.percentile(a_grey, 50)) # Médiane des valeurs de gris des pixels de l'image
        values.append(np.percentile(a_grey, 75)) # Dernier quartile des valeurs de gris des pixels de l'image
        values.append(a_grey.mean()) # Moyenne des valeurs de gris des pixels de l'image
        # Image en nuances de gris après filtre Sobel
        a_sobel = sobel(a_grey)
        values.append(a_sobel.sum()) # Somme des valeurs des pixels
        values.append(np.percentile(a_sobel, 25)) # Premier quartile des valeurs des pixels
        values.append(np.percentile(a_sobel, 50)) # Médiane des valeurs des pixels
        values.append(np.percentile(a_sobel, 75)) # Dernier quartile des valeurs des pixels
        values.append(a_sobel.mean()) # Moyenne des valeurs des pixels
        les_values.append(values)
    les_values = np.array(les_values)
    les_values = les_values.mean(axis=0) # Valeurs moyennes (par catégorie) de toutes les images

    img_test.append(les_values)
    les_index_test.append(int(dossier[4:]))

img_test = np.array(img_test) # Transformation en tableau numpy
les_index_test = np.array(les_index_test)

## Conversion en dataframe

df_img_test = pd.DataFrame(data=img_test,
                            columns=['avg_blue', 'avg_green', 'avg_red',
                                     'sum_grey', 'first_quartile_grey', 'median_grey', 'last_quartile_grey', 'mean_grey',
                                     'sum_sobel', 'first_quartile_sobel', 'median_sobel', 'last_quartile_sobel', 'mean_sobel'])

df_img_test.insert(0, "id", les_index_test)

## Ecriture dans un fichier

df_img_test.to_csv('C:\\Users\leche\Desktop\Data_Challenge\images_test.csv', index=False)



img_train = []
les_index_train = []

for dossier in data_train_list:
    les_images = os.listdir(data_path_train+'/'+dossier)
    les_values = [] # Liste pour ranger les listes de valeurs d'intérêt
    for img in les_images:
        values = [] # Liste des valeurs d'intérêt pour cette image
        # Image en couleur
        a_raw = cv2.imread(data_path_train+'/'+dossier+'/'+img)
        values.append(a_raw[:,:,0].mean()) # Valeur moyenne de bleu sur l'image
        values.append(a_raw[:,:,1].mean()) # Valeur moyenne de vert sur l'image
        values.append(a_raw[:,:,2].mean()) # Valeur moyenne de rouge sur l'image
        # Image en nuances de gris
        a_grey = cv2.cvtColor(a_raw, cv2.COLOR_BGR2GRAY)
        values.append(a_grey.sum()) # Somme des valeurs de gris des pixels de l'image
        values.append(np.percentile(a_grey, 25)) # Premier quartile des valeurs de gris des pixels de l'image
        values.append(np.percentile(a_grey, 50)) # Médiane des valeurs de gris des pixels de l'image
        values.append(np.percentile(a_grey, 75)) # Dernier quartile des valeurs de gris des pixels de l'image
        values.append(a_grey.mean()) # Moyenne des valeurs de gris des pixels de l'image
        # Image en nuances de gris après filtre Sobel
        a_sobel = sobel(a_grey)
        values.append(a_sobel.sum()) # Somme des valeurs des pixels
        values.append(np.percentile(a_sobel, 25)) # Premier quartile des valeurs des pixels
        values.append(np.percentile(a_sobel, 50)) # Médiane des valeurs des pixels
        values.append(np.percentile(a_sobel, 75)) # Dernier quartile des valeurs des pixels
        values.append(a_sobel.mean()) # Moyenne des valeurs des pixels
        les_values.append(values)
    les_values = np.array(les_values)
    les_values = les_values.mean(axis=0) # Valeurs moyennes (par catégorie) de toutes les images

    img_train.append(les_values)
    les_index_train.append(int(dossier[4:]))

img_train = np.array(img_train) # Transformation en tableau numpy
les_index_train = np.array(les_index_train)

df_img_train = pd.DataFrame(data=img_train,
                            columns=['avg_blue', 'avg_green', 'avg_red',
                                     'sum_grey', 'first_quartile_grey', 'median_grey', 'last_quartile_grey', 'mean_grey',
                                     'sum_sobel', 'first_quartile_sobel', 'median_sobel', 'last_quartile_sobel', 'mean_sobel'])

df_img_train.insert(0, "id", les_index_train)

df_img_train.to_csv('path\images_train.csv', index=False)
df_img_test.to_csv('path\images_test.csv', index=False)

In [25]:
images_train=pd.read_csv("images_train.csv")
images_test=pd.read_csv("images_test.csv")

np.shape(images_test)

(9339, 14)

# Méthodes d'apprentissage sur les données images

In [28]:
#decoupage en echantillon train et test 

#découpage en echantillon train et échantillon test 
from sklearn.model_selection import train_test_split


X_train,X_test = train_test_split(images_train, test_size = 0.40)  


## Regression linéaire

In [29]:
reg=LinearRegression().fit(X_train,Y_train)
pred=reg.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']

#score
custom_metric_function(Y_test, pred) # 118.88325501109436

118.88325501109436

## Kneighbors

In [30]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)
pred=knn.predict(X_test)

pred=pd.DataFrame(pred)
pred.columns=['price']

#score
custom_metric_function(Y_test, pred) # 69.69755644299906

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


69.69755644299906

## Random Forest

In [36]:
from sklearn.ensemble import RandomForestRegressor
maxnodes = [2,4,6,8,10,15,20,30,50,100,200,300,400,500,800]
maxfeatures=range(2,10)

oob_error = []

for maxfe in maxfeatures:
    RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)


for maxno in maxnodes:
    RF = RandomForestRegressor(n_estimators=50,max_leaf_nodes=maxno,oob_score = True).fit(X_train, Y_train)

pred=RF.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']

custom_metric_function(Y_test, pred) # 122.2677091178678

/tmp/ipykernel_93/3268112471.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_93/3268112471.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_93/3268112471.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_93/3268112471.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

122.2677091178678

## XGboost


On va tenter une XGBoost uniquement sur la table images_train

In [31]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import xgboost as xgb

xgb=XGBRegressor(objective ='reg:linear',verbosity=0,base_score=1,n_estimators=1000,booster='gblinear',max_depth=5,learning_rate=0.1,loss="mean_absolute_error")
xgb.fit(X_train,Y_train)
xgb_pred_images = xgb.predict(X_test)
xgb_pred_images = pd.DataFrame(xgb_pred_images)


#score
xgb_pred_images.columns=['price']
custom_metric_function(Y_test, xgb_pred_images) #118.76961240341659

118.76961240341659

## Réseau de neurones

In [38]:
#on reprend le modele réalise précédemment 

modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)

training = modele.fit (X_train, Y_train, epochs = 70, validation_split = 0.1)

#historique = pd.DataFrame(training.history)
#historique['epoque'] = training.epoch
#figure, axe = plt.subplots(figsize = (14,8))
#num_epoque = historique.shape[0]
#axe.plot(np.arange(0, num_epoque), historique["mae"],
#label = "Training MAE", lw = 3, color = 'red')
#axe.plot(np.arange(0, num_epoque), historique["val_mae"],
#label = "Validation MAE", lw = 3, color = 'blue')
#axe.legend ()
#plt.tight_layout()
#plt.show()

#test1 = test.iloc[[10]]
test_prediction = modele.predict(X_test).squeeze()
#test_label = test_cible.iloc[10]
#print("Prédiction du modèle = {:.2f}".format(test_prediction))
#print("Valeur actuelle = {:.2f}".format(test_label))

#score = modele.evaluate( X_test, Y_test, verbose = 0)
#print(score)

pred=pd.DataFrame(test_prediction)

pred=pd.DataFrame(pred)
pred.columns=['price']
pred

custom_metric_function(Y_test, pred) #63.26478563675225

Epoch 1/70
421/421 [==============================] - 3s 5ms/step - loss: 75.3091 - val_loss: 59.7175
Epoch 2/70
421/421 [==============================] - 2s 4ms/step - loss: 63.9190 - val_loss: 60.6634
Epoch 3/70
421/421 [==============================] - 2s 4ms/step - loss: 63.2078 - val_loss: 59.6585
Epoch 4/70
421/421 [==============================] - 2s 4ms/step - loss: 63.3078 - val_loss: 60.6383
Epoch 5/70
421/421 [==============================] - 2s 4ms/step - loss: 62.4960 - val_loss: 59.6122
Epoch 6/70
421/421 [==============================] - 2s 4ms/step - loss: 62.6949 - val_loss: 64.0466
Epoch 7/70
421/421 [==============================] - 2s 4ms/step - loss: 62.7633 - val_loss: 60.9974
Epoch 8/70
421/421 [==============================] - 2s 4ms/step - loss: 62.4908 - val_loss: 60.0355
Epoch 9/70
421/421 [==============================] - 2s 4ms/step - loss: 62.4628 - val_loss: 64.6643
Epoch 10/70
421/421 [==============================] - 2s 4ms/step - loss: 62.5337

63.26478563675225

# Méthodes sur images & Variables

In [94]:
#on re-importe les jeux de données contenant toutes les informations au préalable 

X_train=pd.read_csv("X_train.csv")

X_train[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]] = X_train[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]].replace(np.nan,"unknown")
from sklearn import preprocessing 
label = preprocessing.LabelEncoder() 
X_train['property_type' ]=label.fit_transform(X_train['property_type'])
X_train['city' ]=label.fit_transform(X_train['city'])
X_train['energy_performance_category' ]=label.fit_transform(X_train['energy_performance_category'])
X_train['ghg_category']=label.fit_transform(X_train['ghg_category'])
X_train['exposition']=label.fit_transform(X_train['exposition'])

import statistics

X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']] = X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].replace(np.nan,X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].median())
X_train

images_train = pd.read_csv("images_train.csv")

X_train_total = pd.concat([X_train, images_train], axis=1)

# jeu de données test 
X_test[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]] = X_test[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]].replace(np.nan,"unknown")

label = preprocessing.LabelEncoder() 
X_test['property_type' ]=label.fit_transform(X_test['property_type'])
X_test['city' ]=label.fit_transform(X_test['city'])
X_test['energy_performance_category' ]=label.fit_transform(X_test['energy_performance_category'])
X_test['ghg_category']=label.fit_transform(X_test['ghg_category'])
X_test['exposition']=label.fit_transform(X_test['exposition'])

X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']] = X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].replace(np.nan,X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].median())
X_test

images_test = pd.read_csv("images_test.csv")

X_test_total = pd.concat([X_test, images_test], axis=1)




In [47]:
from sklearn.model_selection import train_test_split

X_train,X_test = train_test_split(X_train_total, test_size = 0.60)  


## Regression linéaire

In [48]:
reg=LinearRegression().fit(X_train,Y_train)
pred=reg.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']

#score
custom_metric_function(Y_test, pred) # 116.51311791494314

116.51311791494314

## kneighbors

In [49]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)
pred=knn.predict(X_test)

pred=pd.DataFrame(pred)
pred.columns=['price']

#score
custom_metric_function(Y_test, pred) # 69.32955162180687

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


69.32955162180687

## Random Forest

In [50]:
from sklearn.ensemble import RandomForestRegressor
maxnodes = [2,4,6,8,10,15,20,30,50,100,200,300,400,500,800]
maxfeatures=range(2,10)

oob_error = []

for maxfe in maxfeatures:
    RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)


for maxno in maxnodes:
    RF = RandomForestRegressor(n_estimators=50,max_leaf_nodes=maxno,oob_score = True).fit(X_train, Y_train)

pred=RF.predict(X_test)
pred=pd.DataFrame(pred)
pred.columns=['price']

custom_metric_function(Y_test, pred) # 123.18279009143613

/tmp/ipykernel_93/1778910494.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_93/1778910494.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_93/1778910494.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(n_estimators=50,max_features=maxfe,oob_score = True).fit(X_train, Y_train)
/tmp/ipykernel_93/1778910494.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

123.18279009143613

## XGboost

In [51]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import xgboost as xgb

xgb=XGBRegressor(objective ='reg:linear',verbosity=0,base_score=1,n_estimators=1000,booster='gblinear',max_depth=5,learning_rate=0.1,loss="mean_absolute_error")
xgb.fit(X_train,Y_train)
xgb_pred = xgb.predict(X_test)
xgb_pred = pd.DataFrame(xgb_pred)


#score
xgb_pred.columns=['price']
custom_metric_function(Y_test, xgb_pred) # 116.2842230484237

116.2842230484237

## Réseau de neurones

In [52]:
#on reprend le modele réalise précédemment 

modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)

training = modele.fit (X_train, Y_train, epochs = 70, validation_split = 0.1)

#historique = pd.DataFrame(training.history)
#historique['epoque'] = training.epoch
#figure, axe = plt.subplots(figsize = (14,8))
#num_epoque = historique.shape[0]
#axe.plot(np.arange(0, num_epoque), historique["mae"],
#label = "Training MAE", lw = 3, color = 'red')
#axe.plot(np.arange(0, num_epoque), historique["val_mae"],
#label = "Validation MAE", lw = 3, color = 'blue')
#axe.legend ()
#plt.tight_layout()
#plt.show()

#test1 = test.iloc[[10]]
test_prediction = modele.predict(X_test).squeeze()
#test_label = test_cible.iloc[10]
#print("Prédiction du modèle = {:.2f}".format(test_prediction))
#print("Valeur actuelle = {:.2f}".format(test_label))

#score = modele.evaluate( X_test, Y_test, verbose = 0)
#print(score)

pred=pd.DataFrame(test_prediction)

pred=pd.DataFrame(pred)
pred.columns=['price']
pred

custom_metric_function(Y_test, pred) # 62.80316840239356

Epoch 1/70
421/421 [==============================] - 3s 5ms/step - loss: 68.2401 - val_loss: 59.8310
Epoch 2/70
421/421 [==============================] - 2s 4ms/step - loss: 63.9521 - val_loss: 59.6867
Epoch 3/70
421/421 [==============================] - 2s 4ms/step - loss: 63.5628 - val_loss: 69.2347
Epoch 4/70
421/421 [==============================] - 2s 4ms/step - loss: 63.2896 - val_loss: 59.7420
Epoch 5/70
421/421 [==============================] - 2s 4ms/step - loss: 63.1376 - val_loss: 67.0479
Epoch 6/70
421/421 [==============================] - 2s 4ms/step - loss: 62.8256 - val_loss: 66.0419
Epoch 7/70
421/421 [==============================] - 2s 4ms/step - loss: 62.8648 - val_loss: 61.0178
Epoch 8/70
421/421 [==============================] - 2s 4ms/step - loss: 62.9761 - val_loss: 68.4606
Epoch 9/70
421/421 [==============================] - 2s 4ms/step - loss: 62.7075 - val_loss: 61.3799
Epoch 10/70
421/421 [==============================] - 2s 4ms/step - loss: 62.6117

63.80316840239356

In [42]:
modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)

training = modele.fit (X_train, Y_train, epochs = 70, validation_split = 0.1)

#historique = pd.DataFrame(training.history)
#historique['epoque'] = training.epoch
#figure, axe = plt.subplots(figsize = (14,8))
#num_epoque = historique.shape[0]
#axe.plot(np.arange(0, num_epoque), historique["mae"],
#label = "Training MAE", lw = 3, color = 'red')
#axe.plot(np.arange(0, num_epoque), historique["val_mae"],
#label = "Validation MAE", lw = 3, color = 'blue')
#axe.legend ()
#plt.tight_layout()
#plt.show()

#test1 = test.iloc[[10]]
test_prediction = modele.predict(X_test).squeeze()
#test_label = test_cible.iloc[10]
#print("Prédiction du modèle = {:.2f}".format(test_prediction))
#print("Valeur actuelle = {:.2f}".format(test_label))

#score = modele.evaluate( X_test, Y_test, verbose = 0)
#print(score)

Epoch 1/70
1051/1051 [==============================] - 5s 4ms/step - loss: 66.3999 - val_loss: 60.2688
Epoch 2/70
1051/1051 [==============================] - 3s 3ms/step - loss: 62.9576 - val_loss: 62.7275
Epoch 3/70
1051/1051 [==============================] - 4s 3ms/step - loss: 62.5420 - val_loss: 61.2743
Epoch 4/70
1051/1051 [==============================] - 3s 3ms/step - loss: 62.3169 - val_loss: 61.3320
Epoch 5/70
1051/1051 [==============================] - 3s 3ms/step - loss: 62.3328 - val_loss: 59.4810
Epoch 6/70
1051/1051 [==============================] - 3s 3ms/step - loss: 62.0847 - val_loss: 62.7416
Epoch 7/70
1051/1051 [==============================] - 3s 3ms/step - loss: 61.9943 - val_loss: 63.4805
Epoch 8/70
1051/1051 [==============================] - 3s 3ms/step - loss: 61.9243 - val_loss: 62.8076
Epoch 9/70
1051/1051 [==============================] - 3s 3ms/step - loss: 61.8962 - val_loss: 64.3879
Epoch 10/70
1051/1051 [==============================] - 3s 3ms/

In [21]:
modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train_total.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)

training = modele.fit (X_train_total, Y_train, epochs = 70, validation_split = 0.2)

#historique = pd.DataFrame(training.history)
#historique['epoque'] = training.epoch
#figure, axe = plt.subplots(figsize = (14,8))
#num_epoque = historique.shape[0]
#axe.plot(np.arange(0, num_epoque), historique["mae"],
#label = "Training MAE", lw = 3, color = 'red')
#axe.plot(np.arange(0, num_epoque), historique["val_mae"],
#label = "Validation MAE", lw = 3, color = 'blue')
#axe.legend ()
#plt.tight_layout()
#plt.show()

#test1 = test.iloc[[10]]
test_prediction = modele.predict(X_test_total).squeeze()
#test_label = test_cible.iloc[10]
#print("Prédiction du modèle = {:.2f}".format(test_prediction))
#print("Valeur actuelle = {:.2f}".format(test_label))

#score = modele.evaluate( test, test_cible, verbose = 0)
#print(score)

pred=pd.DataFrame(test_prediction)

pred=pd.DataFrame(pred)
pred.columns=['price']
pred

custom_metric_function(Y_test, pred) #142.3458142115817

Epoch 1/70
935/935 [==============================] - 5s 4ms/step - loss: 68.1442 - val_loss: 60.8239
Epoch 2/70
935/935 [==============================] - 4s 4ms/step - loss: 63.0580 - val_loss: 65.4124
Epoch 3/70
935/935 [==============================] - 3s 4ms/step - loss: 62.8459 - val_loss: 60.8195
Epoch 4/70
935/935 [==============================] - 4s 4ms/step - loss: 62.7411 - val_loss: 61.1039
Epoch 5/70
935/935 [==============================] - 4s 4ms/step - loss: 62.5785 - val_loss: 61.2421
Epoch 6/70
935/935 [==============================] - 4s 4ms/step - loss: 62.3956 - val_loss: 61.0358
Epoch 7/70
935/935 [==============================] - 4s 4ms/step - loss: 62.2601 - val_loss: 59.7426
Epoch 8/70
935/935 [==============================] - 6s 6ms/step - loss: 62.1681 - val_loss: 59.7496
Epoch 9/70
935/935 [==============================] - 7s 7ms/step - loss: 62.0549 - val_loss: 59.8015
Epoch 10/70
935/935 [==============================] - 7s 8ms/step - loss: 62.2886

142.3458142115817

# Estimation du modele intégral 

In [57]:

X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
Y_train = pd.read_csv("y_train.csv")


images_train = pd.read_csv("images_train.csv")
images_test = pd.read_csv("images_test.csv")


X_train[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]] = X_train[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]].replace(np.nan,"unknown")

label = preprocessing.LabelEncoder() 
X_train['property_type' ]=label.fit_transform(X_train['property_type'])
X_train['city' ]=label.fit_transform(X_train['city'])
X_train['energy_performance_category' ]=label.fit_transform(X_train['energy_performance_category'])
X_train['ghg_category']=label.fit_transform(X_train['ghg_category'])
X_train['exposition']=label.fit_transform(X_train['exposition'])


X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']] = X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].replace(np.nan,X_train[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].median())

X_test[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]] = X_test[['property_type' ,'city','energy_performance_category','ghg_category','exposition' ]].replace(np.nan,"unknown")

label = preprocessing.LabelEncoder() 
X_test['property_type' ]=label.fit_transform(X_test['property_type'])
X_test['city' ]=label.fit_transform(X_test['city'])
X_test['energy_performance_category' ]=label.fit_transform(X_test['energy_performance_category'])
X_test['ghg_category']=label.fit_transform(X_test['ghg_category'])
X_test['exposition']=label.fit_transform(X_test['exposition'])

X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']] = X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].replace(np.nan,X_test[['approximate_latitude','approximate_longitude','size','floor','land_size','nb_bedrooms','energy_performance_value','ghg_value','nb_rooms', 
'nb_bedrooms','nb_bathrooms','nb_parking_places','nb_boxes','nb_photos','has_a_balcony','nb_terraces','has_a_cellar','has_a_garage', 'has_air_conditioning','last_floor','upper_floors']].median())


# on combine les données tabulaires aux données images

X_train_total = pd.concat([X_train, images_train], axis=1)
X_test_total = pd.concat([X_test, images_test], axis=1)

print(np.shape(X_train_total))
print(np.shape(X_test_total))

(37368, 41)
(9339, 41)


In [58]:
# on estime le modèle à l'aide d'un réseau de neurones#on reprend le modele réalise précédemment 

modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train_total.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)

training = modele.fit (X_train_total, Y_train, epochs = 70, validation_split = 0.1)

#historique = pd.DataFrame(training.history)
#historique['epoque'] = training.epoch
#figure, axe = plt.subplots(figsize = (14,8))
#num_epoque = historique.shape[0]
#axe.plot(np.arange(0, num_epoque), historique["mae"],
#label = "Training MAE", lw = 3, color = 'red')
#axe.plot(np.arange(0, num_epoque), historique["val_mae"],
#label = "Validation MAE", lw = 3, color = 'blue')
#axe.legend ()
#plt.tight_layout()
#plt.show()

#test1 = test.iloc[[10]]
test_prediction = modele.predict(X_test_total).squeeze()


Epoch 1/70
1051/1051 [==============================] - 6s 4ms/step - loss: 85.3642 - val_loss: 80.6689
Epoch 2/70
1051/1051 [==============================] - 4s 4ms/step - loss: 81.3460 - val_loss: 81.0760
Epoch 3/70
1051/1051 [==============================] - 4s 4ms/step - loss: 81.1384 - val_loss: 79.6387
Epoch 4/70
1051/1051 [==============================] - 4s 4ms/step - loss: 81.0555 - val_loss: 83.0239
Epoch 5/70
1051/1051 [==============================] - 4s 4ms/step - loss: 80.9722 - val_loss: 81.9329
Epoch 6/70
1051/1051 [==============================] - 4s 4ms/step - loss: 80.8833 - val_loss: 80.7364
Epoch 7/70
1051/1051 [==============================] - 4s 4ms/step - loss: 80.9151 - val_loss: 80.6343
Epoch 8/70
1051/1051 [==============================] - 4s 4ms/step - loss: 80.9040 - val_loss: 81.3740
Epoch 9/70
1051/1051 [==============================] - 4s 4ms/step - loss: 80.9165 - val_loss: 81.4550
Epoch 10/70
1051/1051 [==============================] - 4s 4ms/

In [59]:
id_annonce = pd.Series(X_test_total.iloc[:,0])
y_pred = pd.DataFrame(test_prediction,columns=['price'],index=id_annonce)

y_pred1.to_csv('Y_pre.csv',index=True)

,price
id_annonce,
35160615,80028.031250
35830639,150966.406250
36016657,89176.156250
35759225,80057.625000
35252229,77101.742188
...,...
36052217,86968.835938
35823719,89214.085938
35793053,72421.203125


In [99]:
modele = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train_total.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(1)])

optimiseur = tf.keras.optimizers.Adagrad(lr=0.01)
modele.compile(loss = tf.keras.losses.mean_absolute_percentage_error,
optimizer = optimiseur)




training = modele.fit (X_train_total, Y_train, epochs = 30, validation_split = 0.1)

test_prediction = modele.predict(X_test_total)


Epoch 1/30
/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adagrad.py:81: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
1051/1051 [==============================] - 4s 3ms/step - loss: 83.9810 - val_loss: 79.7164
Epoch 2/30
1051/1051 [==============================] - 3s 3ms/step - loss: 80.8830 - val_loss: 81.2636
Epoch 3/30
1051/1051 [==============================] - 3s 3ms/step - loss: 80.6812 - val_loss: 80.6605
Epoch 4/30
1051/1051 [==============================] - 3s 3ms/step - loss: 80.6012 - val_loss: 80.6534
Epoch 5/30
1051/1051 [==============================] - 3s 3ms/step - loss: 80.5428 - val_loss: 80.3564
Epoch 6/30
1051/1051 [==============================] - 3s 3ms/step - loss: 80.5173 - val_loss: 80.5439
Epoch 7/30
1051/1051 [==============================] - 3s 3ms/step - loss: 80.5622 - val_loss: 80.7248
Epoch 8/30
1051/1051 [==============================] - 3s 3ms/

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c590c73-ebe5-4e60-8b74-f7ba67114dc0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>